In [25]:
import pandas as pd
df=pd.read_csv('mental_disorders_reddit.csv')

In [26]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import re
import string

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [27]:
df=df.dropna(how='any')

In [28]:
import nltk
!pip install nltk

In [29]:
def convert_lowercase(text):
    text = text.lower()
    return text

def remove_url(text):
    re_url = re.compile('https?://\S+|www\.\S+')
    return re_url.sub('', text)

exclude = string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

def remove_stopwords(text):
    new_list = []
    words = word_tokenize(text)
    stopwrds = stopwords.words('english')
    for word in words:
        if word not in stopwrds:
            new_list.append(word)
    return ' '.join(new_list)


def perform_stemming(text):
    stemmer = PorterStemmer()
    new_list = []
    words = word_tokenize(text)
    for word in words:
        new_list.append(stemmer.stem(word))

    return " ".join(new_list)


def perform_lemmatize(text):
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    new_list = []
    words = word_tokenize(text)
    for word in words:
        new_list.append(lem.lemmatize(word))

    return " ".join(new_list)

In [30]:
df

,title,selftext,created_utc,over_18,subreddit
0,Life is so pointless without others,Does anyone else think the most important part...,1650356960,False,BPD
1,Cold rage?,Hello fellow friends 😄\n\nI'm on the BPD spect...,1650356660,False,BPD
2,I don’t know who I am,My [F20] bf [M20] told me today (after I said ...,1650355379,False,BPD
3,HELP! Opinions! Advice!,"Okay, I’m about to open up about many things I...",1650353430,False,BPD
4,help,[removed],1650350907,False,BPD
...,...,...,...,...,...
701779,I really need to talk to a therapist..,I can't afford a real session and it's 11 PM. ...,1415332108,False,mentalillness
701781,I have pica,Hello. \n I'm taking steps to get rid ...,1414896638,False,mentalillness
701782,Where can you go to get help for someone menta...,Someone (a war veteran) I know is mentally ill...,1396298261,False,mentalillness
701783,I am rooster illusion,AMA,1344639905,False,mentalillness


In [33]:
df['selftext']=df['selftext'].apply(remove_url)
df['selftext']=df['selftext'].apply(remove_punc)
df['selftext']=df['selftext'].apply(convert_lowercase)
df['selftext']=df['selftext'].apply(remove_stopwords)


df['title']=df['title'].apply(remove_url)
df['title']=df['title'].apply(remove_punc)
df['title']=df['title'].apply(convert_lowercase)
df['title']=df['title'].apply(remove_stopwords)





In [70]:
df['subreddit'].value_counts()

BPD              233119
Anxiety          167032
depression       156708
bipolar           46666
mentalillness     44249
schizophrenia     20280
Name: subreddit, dtype: int64

In [155]:

def category_equalization(data_df,col):# функция для баланса значений категорий в таргет 
    for value in list(data_df[col].unique()):
        
        num_rows_to_remove=data_df[col].value_counts()[value]-data_df[col].value_counts()['schizophrenia']#надо поменять на мимнимальное значение в value_counts 
        
        rows_with_value = data_df[data_df[col] == value]

        data_df = data_df[~data_df.isin(rows_with_value.head(num_rows_to_remove))]
    return data_df

In [156]:
df_equals=category_equalization(df,"subreddit")

In [157]:
df_equals=df_equals.dropna()

In [158]:
df_equals["subreddit"].value_counts()

BPD              20280
bipolar          20280
depression       20280
Anxiety          20280
schizophrenia    20280
mentalillness    20280
Name: subreddit, dtype: int64

In [159]:
df_sample=df_equals.sample(n=10000, random_state=0)

In [160]:
df_sample["subreddit"].value_counts()

mentalillness    1732
BPD              1688
schizophrenia    1676
Anxiety          1663
bipolar          1627
depression       1614
Name: subreddit, dtype: int64

In [161]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_hist_gradient_boosting
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
import numpy as np

In [170]:


X = df_sample['selftext']
y = df_sample["subreddit"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

pipe_logreg = Pipeline([
           ('vect', TfidfVectorizer()),
           ('clf', LogisticRegression()),
])
parameters = [{
    'vect__max_features':[None, 5000, 10000, 50000],
     'vect__ngram_range': [(1,1), (1,2), (1,3)],
    'clf__penalty': ('l1','l2', 'elasticnet',None),
    'clf__C': (100, 10, 1.0, 0.1, 0.01)
}]

grid_search = GridSearchCV(pipe_logreg, parameters,verbose=5,n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
{'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__max_features': None, 'vect__ngram_range': (1, 1)}
0.574


In [172]:
X = df_equals['selftext']
y = df_equals["subreddit"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


pipe_log = Pipeline([
           ('vect', TfidfVectorizer(max_features= 5000,ngram_range=(1,1))),
           ('clf', LogisticRegression(C=1.0,penalty='l2',random_state=42,n_jobs=-1)),
])
pipe_log.fit(X_train, y_train)
y_pred = pipe_log.predict(X_test)
predicted_prob = pipe_log.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy %s 0.6265614727153189
               precision    recall  f1-score   support

      Anxiety       0.73      0.75      0.74      4041
          BPD       0.75      0.77      0.76      4138
      bipolar       0.72      0.51      0.59      4092
   depression       0.61      0.50      0.55      4125
mentalillness       0.52      0.45      0.48      3972
schizophrenia       0.50      0.79      0.61      3968

     accuracy                           0.63     24336
    macro avg       0.64      0.63      0.62     24336
 weighted avg       0.64      0.63      0.62     24336



In [173]:
pipe_cat = Pipeline([
           ('vect', TfidfVectorizer(max_features=5000,ngram_range=(1,2))),
           ('clf', CatBoostClassifier()),
])
pipe_cat.fit(X_train, y_train)
y_pred = pipe_cat.predict(X_test)
predicted_prob = pipe_cat.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

Learning rate set to 0.099937
0:	learn: 1.7053651	total: 5.01s	remaining: 1h 23m 24s
1:	learn: 1.6437596	total: 9.09s	remaining: 1h 15m 37s
2:	learn: 1.6019494	total: 12.7s	remaining: 1h 10m 12s
3:	learn: 1.5681468	total: 15.9s	remaining: 1h 6m 10s
4:	learn: 1.5399438	total: 19.1s	remaining: 1h 3m 26s
5:	learn: 1.5157747	total: 22.7s	remaining: 1h 2m 39s
6:	learn: 1.4931626	total: 26.7s	remaining: 1h 3m 10s
7:	learn: 1.4746128	total: 30.9s	remaining: 1h 3m 48s
8:	learn: 1.4591452	total: 34.8s	remaining: 1h 3m 49s
9:	learn: 1.4448951	total: 38.2s	remaining: 1h 2m 59s
10:	learn: 1.4321282	total: 41.7s	remaining: 1h 2m 31s
11:	learn: 1.4202062	total: 45.2s	remaining: 1h 2m
12:	learn: 1.4082612	total: 49.6s	remaining: 1h 2m 44s
13:	learn: 1.3987996	total: 53.5s	remaining: 1h 2m 45s
14:	learn: 1.3888618	total: 56.6s	remaining: 1h 1m 58s
15:	learn: 1.3810731	total: 59.8s	remaining: 1h 1m 17s
16:	learn: 1.3721528	total: 1m 2s	remaining: 1h 30s
17:	learn: 1.3633906	total: 1m 6s	remaining: 1h 1

In [174]:
X = df_equals['selftext']
y = df_equals["subreddit"].apply(perform_lemmatize)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


pipe_log = Pipeline([
           ('vect', TfidfVectorizer(max_features= 5000,ngram_range=(1,1))),
           ('clf', LogisticRegression(C=1.0,penalty='l2',random_state=42,n_jobs=-1)),
])
pipe_log.fit(X_train, y_train)
y_pred = pipe_log.predict(X_test)
predicted_prob = pipe_log.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy %s 0.6265614727153189
               precision    recall  f1-score   support

      Anxiety       0.73      0.75      0.74      4041
          BPD       0.75      0.77      0.76      4138
      bipolar       0.72      0.51      0.59      4092
   depression       0.61      0.50      0.55      4125
mentalillness       0.52      0.45      0.48      3972
schizophrenia       0.50      0.79      0.61      3968

     accuracy                           0.63     24336
    macro avg       0.64      0.63      0.62     24336
 weighted avg       0.64      0.63      0.62     24336



In [176]:
df_equals

,title,selftext,created_utc,over_18,subreddit
219059,bpddepression self careneglectbasic care,hey yall today wanted make post basic care one...,1.508618e+09,False,BPD
219060,fp decided ’ better part ways,knew coming actually happened feel like shit y...,1.508617e+09,False,BPD
219061,someone asks,feeling today lately get flashback good things...,1.508612e+09,False,BPD
219062,work,yet ive found position jobless job really like...,1.508612e+09,False,BPD
219063,friends go something without,find snapchat wouldnt anyway probably thought ...,1.508609e+09,False,BPD
...,...,...,...,...,...
701779,really need talk therapist,cant afford real session 11 pm somebody please...,1.415332e+09,False,mentalillness
701781,pica,hello im taking steps get rid pica ive picaing...,1.414897e+09,False,mentalillness
701782,go get help someone mentally ill psyche wards ...,someone war veteran know mentally ill wont see...,1.396298e+09,False,mentalillness
701783,rooster illusion,ama,1.344640e+09,False,mentalillness


In [178]:
df_equals=df_equals[df_equals['selftext'] != 'removed']
df_equals['subreddit'].value_counts()

BPD              20019
Anxiety          19651
mentalillness    18526
bipolar          15666
depression       15628
schizophrenia    11727
Name: subreddit, dtype: int64

In [179]:
df_equals=category_equalization(df_equals,'subreddit')
df_equals=df_equals.dropna()

In [182]:
df_equals['subreddit'].value_counts()

BPD              11727
bipolar          11727
depression       11727
Anxiety          11727
schizophrenia    11727
mentalillness    11727
Name: subreddit, dtype: int64

In [183]:
X = df_equals['selftext']
y = df_equals["subreddit"].apply(perform_lemmatize)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
pipe_log.fit(X_train, y_train)

Pipeline(steps=[('vect', TfidfVectorizer(max_features=5000)),
                ('clf', LogisticRegression(n_jobs=-1, random_state=42))])

accuracy %s 0.6629005897818517
               precision    recall  f1-score   support

      Anxiety       0.72      0.73      0.73      2340
          BPD       0.72      0.76      0.74      2316
      bipolar       0.71      0.65      0.68      2341
   depression       0.59      0.70      0.64      2362
mentalillness       0.53      0.44      0.48      2325
schizophrenia       0.68      0.71      0.69      2389

     accuracy                           0.66     14073
    macro avg       0.66      0.66      0.66     14073
 weighted avg       0.66      0.66      0.66     14073



In [ ]:
y_pred = pipe_log.predict(X_test)
predicted_prob = pipe_log.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

In [215]:
cat_pipeline = Pipeline(steps=[
    ('vect',TfidfVectorizer(max_features= 10000,ngram_range=(1,2)))
])

num_pipeline = Pipeline(steps=[
    
    ('vect', TfidfVectorizer(max_features= 10000,ngram_range=(1,2)))
])

col_trans = ColumnTransformer(transformers=[
   
    ('num_pipeline',num_pipeline,'selftext'),
    ('cat_pipeline',cat_pipeline,'title')
    ],
    
    n_jobs=-1)

In [216]:
pipeline = Pipeline(steps=[('preprocessor', col_trans), ('classifier',LogisticRegression(C=1.0,penalty='l2',random_state=42,n_jobs=-1))])

In [221]:
df_equals['selftext']=df_equals['selftext'].apply(perform_lemmatize)

In [222]:
df_equals['title']=df_equals['title'].apply(perform_lemmatize)

In [223]:
X = df_equals[['selftext','title']]
y = df_equals["subreddit"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('vect',
                                                                   TfidfVectorizer(max_features=10000,
                                                                                   ngram_range=(1,
                                                                                                2)))]),
                                                  'selftext'),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('vect',
                                                                   TfidfVectorizer(max_features=10000,
                                                                                   ngram_range=(1,
                                                                                                2)))]),
                                                  'title')])),
                ('classifier', LogisticRegression(n_jobs=-1, random_state=42))])

In [224]:
y_pred = pipeline.predict(X_test)
predicted_prob = pipeline.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy %s 0.6965110495274639
               precision    recall  f1-score   support

      Anxiety       0.77      0.76      0.77      2340
          BPD       0.75      0.78      0.77      2316
      bipolar       0.75      0.69      0.72      2341
   depression       0.61      0.72      0.66      2362
mentalillness       0.55      0.49      0.52      2325
schizophrenia       0.74      0.73      0.74      2389

     accuracy                           0.70     14073
    macro avg       0.70      0.70      0.70     14073
 weighted avg       0.70      0.70      0.70     14073

